# **Object Detection with Deep Neural Network**

**Prepare environment**

In [ ]:
#install requirenments
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install 'prompt-toolkit==1.0.15'

In [ ]:
#clone tensorflow models to local machine
!git clone https://github.com/tensorflow/models.git

In [ ]:
#set environment
%cd /content/models/research
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%cd /content/models/research

In [ ]:
!python object_detection/builders/model_builder_test.py

**Mount GoogleDrive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive_')

In [ ]:
#check mounted gdrive
%cd /content/gdrive_/My Drive/dave_grohl_detection
!ls

**Installing pycocotools**

In [ ]:
%cd /content/gdrive_/My Drive/dave_grohl_detection/object_detection/metrics
!ls

In [ ]:
!git clone https://github.com/cocodataset/cocoapi


In [ ]:
%cd /content/gdrive_/My Drive/dave_grohl_detection/object_detection/metrics/cocoapi/PythonAPI
!ls

In [ ]:
!python3 setup.py install

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
rm -rf build

**Launch Tensorboard**

In [ ]:
%cd /content/gdrive_/My Drive/dave_grohl_detection
!ls

In [ ]:
LOG_DIR = 'models/model/train'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

**Train Model**

In [ ]:
!python3 model_main.py \
    --pipeline_config_path=models/model/pipeline.config \
    --model_dir=models/model/train \
    --num_train_steps=50000 \
    --num_eval_steps=50000 \
    --alsologtostderr

**Exporting the model for inference**

In [ ]:
#creating frozen graph
!python object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path=models/model/pipeline.config \
--trained_checkpoint_prefix=models/model/train/your_checkpoint
--output_directory=output

In [ ]:
#running object detection
!python object_detection/inference.py \
--input_dir=test_images \
--output_dir=output \
--label_map=data/label_map.pbtxt \
--frozen_graph=output/frozen_inference_graph.pb \
--num_output_classes=1